In [1]:
import os
import pickle
import json
import random

fin = open('/home/mcwave/code/automath/atp/datasets/prover_training_data_6.json', "r")
data = json.load(fin)
print("Data loaded")
data_2 = []

for example in data:
#     print(example)
#     break
#     example = list(example)
    example[-1] = example[-1].split("\n")[0]
#     print(example[-1])
#     break
    data_2.append(tuple(example))

random.shuffle(data_2)
data = data_2
print(len(data), "examples loaded")

Data loaded
4702639 examples loaded


In [2]:
# import datasets

# data = datasets.load_from_disk("datasets/reprover_dataset.dataset")
# data_2 = []
# for i in range(len(data)):
#     data_2.append((data[i]["instruction"], data[i]["response"]))
# data = data_2

In [3]:
from transformers import AutoTokenizer

tokenizer_name = "morph-labs/morph-prover-v0-7b" #"internlm/internlm2-math-7b" #"ScalableMath/Lean-STaR-plus"  # 'Saisam/gpt-neo-math-small' #

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.padding_side = "right"

# Define the separator token
sep_token = "<sep>"
pad_token = "<pad>"
eos_token = "<end>"

# Check if the separator token already exists in the vocabulary
if sep_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([sep_token])
if pad_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([pad_token])

# Set the separator token
tokenizer.sep_token = sep_token
tokenizer.pad_token = pad_token

tokenizer.add_special_tokens({
    'sep_token': sep_token,
    'pad_token': pad_token,
    'eos_token': eos_token
})
# tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
# tokenizer.sep_token = tokenizer.pad_token

3

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from tqdm import tqdm
import os

class InstructionResponseDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instruction, response = self.data[idx]
        
        input_encoding = self.tokenizer(instruction, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        output_encoding = self.tokenizer(response, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        input_mask = input_encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_mask,
            "labels": output_encoding["input_ids"].squeeze()
        }

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Initialize tokenizer and model
# model_name = "models/flan_base/trained_model_600k"  # You can change this to any other Seq2Seq model
model_name = "google/byt5-base"
# tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# model = torch.load(model_name)

# # Prepare your data
# data = [
#     ("Write a poem about spring.", "Blossoms bloom in gentle breeze,\nSunshine warms the earth with ease.\nBirds return with joyful song,\nDays grow longer, winter's gone."),
#     ("Explain photosynthesis.", "Photosynthesis is the process by which plants use sunlight, water, and carbon dioxide to produce oxygen and energy in the form of sugar."),
#     ("What is the capital of France?", "The capital of France is Paris."),
#     ("Describe the water cycle.", "The water cycle involves evaporation, condensation, precipitation, and collection of water on Earth's surface and in the atmosphere."),
#     # Add more instruction-response pairs here
# ]

# Create full dataset
full_dataset = InstructionResponseDataset(data, tokenizer)

# Split into train and test sets
train_size = int(0.9 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

NameError: name 'data' is not defined

In [5]:
for batch in train_dataloader:
    input_ids = batch["input_ids"][0]
    labels = batch["labels"][0]
    for i in range(len(input_ids)):
        print(input_ids[i], labels[i])
    print(tokenizer.decode(input_ids))
    print(tokenizer.decode(labels))
    break

tensor(1) tensor(1)
tensor(1222) tensor(24671)
tensor(264) tensor(307)
tensor(28723) tensor(1122)
tensor(28708) tensor(28734)
tensor(13) tensor(307)
tensor(28980) tensor(1122)
tensor(714) tensor(28740)
tensor(5707) tensor(32001)
tensor(332) tensor(32001)
tensor(28730) tensor(32001)
tensor(28740) tensor(32001)
tensor(13) tensor(32001)
tensor(28948) tensor(32001)
tensor(714) tensor(32001)
tensor(5707) tensor(32001)
tensor(332) tensor(32001)
tensor(28730) tensor(32001)
tensor(28750) tensor(32001)
tensor(13) tensor(32001)
tensor(29152) tensor(32001)
tensor(714) tensor(32001)
tensor(5707) tensor(32001)
tensor(332) tensor(32001)
tensor(28730) tensor(32001)
tensor(28770) tensor(32001)
tensor(13) tensor(32001)
tensor(4138) tensor(32001)
tensor(229) tensor(32001)
tensor(159) tensor(32001)
tensor(160) tensor(32001)
tensor(30047) tensor(32001)
tensor(714) tensor(32001)
tensor(6611) tensor(32001)
tensor(4423) tensor(32001)
tensor(10571) tensor(32001)
tensor(28705) tensor(32001)
tensor(28948) tenso

In [7]:
num_epochs = 10
num_its = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        num_its += 1
        if num_its % 1000 == 0:
            print(f"{num_its} iterations, loss={total_loss/num_its}")
        if num_its % 20000 == 0:
            save_path = f"./models/flan_base/trained_model_{600000 + num_its}"
#             os.makedirs(save_path, exist_ok=True)
            model.save_pretrained(save_path)
    
    # Save the model after each epoch
#     save_path = f"./models/flan_base/trained_model_epoch_{epoch + 1}"
#     os.makedirs(save_path, exist_ok=True)
#     model.save_pretrained(save_path)
#     torch.save(model, save_path)
#     tokenizer.save_pretrained(save_path)
    
    avg_train_loss = total_loss / len(train_dataloader)
    avg_test_loss = evaluate(model, test_dataloader, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Average Train Loss: {avg_train_loss:.4f}")
    print(f"Average Test Loss: {avg_test_loss:.4f}")

Epoch 1/10:   0%|                      | 1000/423238 [13:33<93:15:40,  1.26it/s]

1000 iterations, loss=0.013483682553749531


Epoch 1/10:   0%|                      | 2000/423238 [26:56<93:25:23,  1.25it/s]

2000 iterations, loss=0.013475905947387219


Epoch 1/10:   1%|▏                     | 3000/423238 [40:35<92:40:40,  1.26it/s]

...

Epoch 1/10:  25%|████▏            | 104000/423238 [22:48:20<69:55:56,  1.27it/s]

104000 iterations, loss=0.013562135885671104


Epoch 1/10:  25%|████▏            | 105000/423238 [23:01:30<72:28:17,  1.22it/s]

105000 iterations, loss=0.013557707087904037


Epoch 1/10:  25%|████▏            | 105210/423238 [23:04:20<69:44:33,  1.27it/s]

KeyboardInterrupt



In [ ]:
torch.save(model, "trained_model_697k")

In [ ]:
torch.save(model, "trained_model_reproverfinetuned")